In [20]:

#!/usr/bin/env python
import pandas as pd 
import datetime as dt 
import os 
from wmf import wmf
from multiprocessing import Pool
import numpy as np
import pickle 
import alarmas as al
import glob 
import time
import pylab as pl
import json
import datetime
import warnings
warnings.filterwarnings('ignore')

In [2]:
ruta_configuracion_1 = '/media/nicolas/Home/Jupyter/Soraya/git/Alarmas/03_modelo/Op_AMVA60m/configfile.md'
RutasList = al.get_rutesList(ruta_configuracion_1)

# Lee rutas de objetos de entrada
ruta_cuenca = al.get_ruta(RutasList, 'ruta_cuenca')
ruta_campos = al.get_ruta(RutasList, 'ruta_campos')
# Lee rutas de salida - lluvia radar
ruta_out_rain = al.get_ruta(RutasList, 'ruta_rain')
ruta_out_rain_png = al.get_ruta(RutasList, 'ruta_rain_png')

cuenca = wmf.SimuBasin(rute = ruta_cuenca, SimSlides = True)

rutaConfig=ruta_configuracion_1

In [11]:
rutaRes=ruta_out_rain+'Lluvia_historica'
rutaRes

'/media/nicolas/Home/Jupyter/Soraya/git/Alarmas/03_modelo/Op_AMVA60m/results/01_rain/Lluvia_historica'

cuenca.rain_radar2basin_from_array(status='old',ruta_out= rutaRes)

In [10]:
def rain_radar2basin_from_array(self,vec=None,ruta_out=None,fecha=None,dt=None,
    status='update',umbral = 0.01, doit = False):
    'Descripcion: Genera campos de lluvia a partir de archivos array\n'\
    '\n'\
    'Parametros\n'\
    '----------\n'\
    'self : .\n'\
    'vec: Array en forma de la cuenca con la informacion.\n'\
    'ruta_out: Ruta donde escribe el binario con la lluvia.\n'\
    'fecha: Fecha del registro actual.\n'\
    'dt: Intervalo de tiempo entre registros.\n'\
    'status: Estado en el cual se encuentra el binario que se va a pasar a campo.\n'\
    '   update: (Defecto) Con este estado se genera un binario y se agregan nuevos campos.\n'\
    '   old: Estado para abrir y tomar las propiedades de self.radar.. para la generacion de un binario.\n'\
    '   close: Cierra un binario que se ha generado mediante update.\n'\
    '   reset: Reinicia las condiciones de self.radar... para la creacion de un campo nuevo.\n'\
    'doit: Independiente del umbral escribe el binario en la siguiente entrada.\n'\
    'Retornos\n'\
    '----------\n'\
    'Guarda el binario, no hay retorno\n'\
    'meanRain :  La serie de lluvia promedio.\n'\
    '\n'\
    'Mirar Tambien\n'\
    '----------\n'\
    'rain_interpolate_idw: interpola campos mediante la metodologia idw.\n'\
    'rain_radar2basin_from_asc: Mete campos de lluvia mediante multiples arrays.\n'\
    #Edita la ruta de salida 
    if ruta_out  is not  None:
        if ruta_out.endswith('.hdr') or ruta_out.endswith('.bin'):
            ruta_bin = ruta_out[:-3]+'.bin'
            ruta_hdr = ruta_out[:-3]+'.hdr'
        else:
            ruta_bin = ruta_out+'.bin'
            ruta_hdr = ruta_out+'.hdr'
    #Establece la cantidad de elementos de acuerdo al tipo de cuenca
    if self.modelType[0] is 'c':
        N = self.ncells
    elif self.modelType[0] is 'h':
        N = self.nhills
        try:
            if vec.shape[0]  == self.ncells:
                vec = self.Transform_Basin2Hills(vec,sumORmean=1)
        except:
            pass
    # De acerudo al estado actualiza las variables o guarda el 
    # binario final 
    actualizo = 1
    if status == 'update':
        #Entrada 1 es la entrada de campos sin lluvia 
        if len(self.radarDates) == 0:
            models.write_int_basin(ruta_bin,np.zeros((1,N)),1,N,1)
        if vec.mean() > umbral or doit:
            #Actualiza contador, lluvia media y pocisiones 
            self.radarCont +=1
            self.radarMeanRain.append(vec.mean())
            self.radarPos.append(self.radarCont)
            #Guarda el vector 
            vec = vec*1000; vec = vec.astype(int)
            models.write_int_basin(ruta_bin,np.zeros((1,N))+vec,
                self.radarCont,N,1)
            actualizo = 0
        else:
            #lluvia media y pocisiones 
            self.radarMeanRain.append(0.0)
            self.radarPos.append(1)
        self.radarDates.append(fecha)
    #Si ya no va a agregar nada, no agrega mas campos y genera el .hdr 
    elif status == 'close':
        self.radarMeanRain = np.array(self.radarMeanRain)
        self.radarPos = np.array(self.radarPos)
        #Guarda un archivo con informacion de la lluvia 
        f=open(ruta_hdr[:-3]+'hdr','w')
        f.write('Numero de celdas: %d \n' % self.ncells)
        f.write('Numero de laderas: %d \n' % self.nhills)
        f.write('Numero de registros: %d \n' % self.radarMeanRain.shape[0])
        f.write('Numero de campos no cero: %d \n' % self.radarPos.max())
        f.write('Tipo de interpolacion: radar \n')
        f.write('IDfecha, Record, Lluvia, Fecha \n')
        c = 1
        for d,pos,m in zip(self.radarDates,
            self.radarPos,self.radarMeanRain):
            f.write('%d, \t %d, \t %.2f, %s \n' % (c,pos,m,d.strftime('%Y-%m-%d-%H:%M')))
            c+=1
        f.close()
        #Vuelve las variables listas de nuevo 
        self.radarMeanRain = self.radarMeanRain.tolist()
        self.radarPos = self.radarPos.tolist()
    elif status == 'reset':
        #Variables de radar
        self.radarDates = []
        self.radarPos = []
        self.radarMeanRain = []
        self.radarCont = 1
    elif status == 'old':
        #si es un archivo viejo, lo abre para tomar las variables y continuar en ese punto 
        f=open(ruta_hdr[:-3]+'hdr','r')
        Lista = f.readlines()
        self.radarCont = int(Lista[3].split()[-1])
        cantidadIds = int(Lista[2].split()[-1])
        f.close()
        #Abre con numpy para simplificar las cosas 
        a = np.loadtxt(ruta_hdr,skiprows=6,dtype='str').T
        if self.radarCont >= 1 and cantidadIds > 1:
            self.radarPos = [int(i.split(',')[0]) for i in a[1]]
            self.radarMeanRain = [float(i.split(',')[0]) for i in a[2]]
            for i in a[3]:
                d = datetime.datetime.strptime(i,'%Y-%m-%d-%H:%M')
                self.radarDates.append(d)
        else:
            self.radarPos = [int(a[1].split(',')[0])]
            self.radarMeanRain = [float(a[2].split(',')[0])]
            self.radarDates = [datetime.datetime.strptime(a[-1], '%Y-%m-%d-%H:%M')]
    return actualizo 

In [27]:
rutaRes

'/media/nicolas/Home/Jupyter/Soraya/git/Alarmas/03_modelo/Op_AMVA60m/results/01_rain/Lluvia_historica'

In [23]:
rutebin=ruta_out_rain+'Lluvia_actual.bin'

In [25]:
v,r = wmf.models.read_int_basin(rutebin,1, cuenca.ncells)

In [31]:
rutehdr=ruta_out_rain+'Lluvia_actual.hdr'
p=wmf.read_mean_rain(rutehdr)
p.index[0]

Timestamp('2018-01-23 13:35:00')

In [32]:
rain_radar2basin_from_array(cuenca,vec=v,ruta_out=rutaRes,fecha=p.index[0],dt=300,
    status='old',umbral = 0.01, doit = False)

1